# Preprocessing & Constructing new data from all `csv` files
- The aim of this notebook is to make use of every given dataframe sheet
- Features of importance higher than certain threshold were selceted and integrated into the train data
- For the test data, only the selected features in the training dataset were correspondingly added 

In [ ]:
!pip install pandas matplotlib -q
!pip install numpy -q
!pip install dask[dataframe] -q
!pip install "dask[diagnostics]" -qprint(df)print(df)

In [32]:
import pandas as pd

In [72]:
# df_description = pd.read_csv("./data/HomeCredit_columns_description.csv")
# df_sample_submission = pd.read_csv("/kaggle/input/home-credit-default-risk/home-credit-default-risk/sample_submission.csv")
df_application_train = pd.read_csv("./data/application_train.csv")
df_bureau_balance = pd.read_csv("./data/bureau_balance.csv")
df_bureau = pd.read_csv("./data/bureau.csv")
df_credit_card_balance = pd.read_csv("./data/credit_card_balance.csv")
df_installments_payments = pd.read_csv("./data/installments_payments.csv")
df_POS_CASH_balance = pd.read_csv("./data/POS_CASH_balance.csv")
df_previous_application = pd.read_csv("./data/previous_application.csv")

In [73]:
df_application_test = pd.read_csv("./data/application_test.csv")

In [74]:
# Drop columns with all NaN values
df_application_train = df_application_train.dropna(axis=1, how='all')
    
# Drop columns with more than 95% NaN values
threshold = len(df_application_train) * 0.95
df_application_train = df_application_train.dropna(axis=1, thresh=threshold)
    
# Drop NaN values
df_application_train = df_application_train.dropna()
        
# One-hot encode categorical variables
df_application_train = pd.get_dummies(df_application_train)

# Calculate the correlation matrix
corr_matrix = df_application_train.corr()
    
# Get the absolute correlation values with the target variable
important_features = corr_matrix['TARGET'].abs().sort_values(ascending=False)
    
# Filter features with relevance value higher than 0.03
selected_features = important_features[important_features > 0.03]

selected_feature_names = selected_features.index.tolist()

selected_feature_names.append("SK_ID_CURR")
        
# Create a new dataframe with the selected features
df_application_train_selected_features = df_application_train[selected_feature_names]

In [75]:
df_application_train_selected_features.shape

(304531, 28)

In [76]:
df_application_train.shape

(304531, 159)

In [77]:
# Drop columns with all NaN values
df_application_test = df_application_test.dropna(axis=1, how='all')
    
# Drop columns with more than 95% NaN values
threshold = len(df_application_test) * 0.95
df_application_test = df_application_test.dropna(axis=1, thresh=threshold)
    
# Drop NaN values
df_application_test = df_application_test.dropna()
        
# One-hot encode categorical variables
df_application_test = pd.get_dummies(df_application_test)

selected_feature_names.remove("TARGET")

df_application_test_selected_features = df_application_test[selected_feature_names]

print(df_application_test_selected_features.shape)

(47772, 27)


In [78]:
# List of dataframes to process
dataframes = [
    df_bureau,
    df_bureau_balance,
    df_credit_card_balance,
    df_installments_payments,
    df_POS_CASH_balance,
    df_previous_application,
]

In [79]:
def process_dataframe_train(df, merged_df):
    # Check if 'SK_ID_CURR' column exists in the dataframe
    if 'SK_ID_CURR' not in df.columns:
        return merged_df
    
    # Drop columns with all NaN values
    df = df.dropna(axis=1, how='all')
    
    # Drop columns with more than 95% NaN values
    threshold = len(df) * 0.95
    df = df.dropna(axis=1, thresh=threshold)
    
    # Drop NaN values
    df = df.dropna()
    
    df = df.drop_duplicates(subset=['SK_ID_CURR'])
        
    # One-hot encode categorical variables
    df = pd.get_dummies(df)
    
    df = pd.merge(df, df_application_train_selected_features[['SK_ID_CURR', 'TARGET']], on='SK_ID_CURR', how='left')
    
    # Calculate the correlation matrix
    corr_matrix = df.corr()
    
    # Get the absolute correlation values with the target variable
    important_features = corr_matrix['TARGET'].abs().sort_values(ascending=False)
    
    # Filter features with relevance value higher than 0.06
    selected_features = important_features[important_features > 0.06]
        
    # Extract the feature names
    selected_feature_names = selected_features.index.tolist()
    
    # Ensure 'SK_ID_CURR' is included in the selected features
    if 'SK_ID_CURR' not in selected_feature_names:
        selected_feature_names.append('SK_ID_CURR')
        
    selected_feature_names.remove("TARGET")
    selected_feature_names = list(set(selected_feature_names))
        
    # Create a new dataframe with the selected features
    df_selected_features = df[selected_feature_names]
    
    # Merge with the existing merged_df on 'SK_ID_CURR'
    merged_df = pd.merge(merged_df, df_selected_features, on='SK_ID_CURR', how='left')
    
    return merged_df

In [105]:
def process_dataframe_test(df, merged_df, df_train):
    # Check if 'SK_ID_CURR' column exists in the dataframe
    if 'SK_ID_CURR' not in df.columns:
        return merged_df
    
    # Drop columns with all NaN values
    df = df.dropna(axis=1, how='all')
    
    # Drop columns with more than 95% NaN values
    threshold = len(df) * 0.95
    df = df.dropna(axis=1, thresh=threshold)
    
    # Drop NaN values
    df = df.dropna()
    
    df = df.drop_duplicates(subset=['SK_ID_CURR'])
        
    # One-hot encode categorical variables
    df = pd.get_dummies(df)
    
    selected_feature_names = df_train.columns.tolist()
    
    # Ensure 'SK_ID_CURR' is included in the selected features
    if 'SK_ID_CURR' not in selected_feature_names:
        selected_feature_names.append('SK_ID_CURR')
        
    selected_feature_names.remove("TARGET")
    
    intersection_columns = list(set(df.columns) & set(selected_feature_names))
        
    # Create a new dataframe with the selected features
    df_selected_features = df[intersection_columns]

    difference_columns = list(set(merged_df.columns) - set(df_selected_features.columns))

    difference_columns.append("SK_ID_CURR")
    
    # Merge with the existing merged_df on 'SK_ID_CURR'
    merged_df = pd.merge(merged_df[difference_columns], df_selected_features, on='SK_ID_CURR', how='left')
    
    return merged_df

In [81]:
# Initialize the merged dataframe with df_application_train_not_na_onehot
df_train = df_application_train_selected_features.copy()

# Process each dataframe and merge
for df in dataframes:
    df_train = process_dataframe_train(df, df_train)

In [82]:
len(df_train.columns)

33

In [83]:
df_train.columns

Index(['TARGET', 'EXT_SOURCE_2', 'DAYS_BIRTH', 'REGION_RATING_CLIENT_W_CITY',
       'REGION_RATING_CLIENT', 'NAME_INCOME_TYPE_Working',
       'NAME_EDUCATION_TYPE_Higher education', 'DAYS_LAST_PHONE_CHANGE',
       'CODE_GENDER_M', 'CODE_GENDER_F', 'DAYS_ID_PUBLISH',
       'REG_CITY_NOT_WORK_CITY',
       'NAME_EDUCATION_TYPE_Secondary / secondary special',
       'NAME_INCOME_TYPE_Pensioner', 'ORGANIZATION_TYPE_XNA', 'FLAG_EMP_PHONE',
       'DAYS_EMPLOYED', 'REG_CITY_NOT_LIVE_CITY', 'FLAG_DOCUMENT_3',
       'DAYS_REGISTRATION', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE',
       'LIVE_CITY_NOT_WORK_CITY', 'DEF_30_CNT_SOCIAL_CIRCLE',
       'DEF_60_CNT_SOCIAL_CIRCLE', 'AMT_CREDIT',
       'NAME_HOUSING_TYPE_With parents', 'SK_ID_CURR', 'DAYS_CREDIT',
       'AMT_BALANCE', 'AMT_TOTAL_RECEIVABLE', 'AMT_RECIVABLE',
       'AMT_RECEIVABLE_PRINCIPAL'],
      dtype='object')

In [106]:
# Initialize the merged dataframe with df_application_train_not_na_onehot
df_test = df_application_test_selected_features.copy()

# Process each dataframe and merge
for df in dataframes:
    df_test = process_dataframe_test(df, df_test, df_train)

In [107]:
len(df_test.columns)

32

In [108]:
df_test.columns

Index(['DAYS_CREDIT', 'REGION_RATING_CLIENT',
       'NAME_EDUCATION_TYPE_Secondary / secondary special',
       'DEF_30_CNT_SOCIAL_CIRCLE', 'AMT_RECIVABLE', 'NAME_INCOME_TYPE_Working',
       'AMT_RECEIVABLE_PRINCIPAL', 'NAME_EDUCATION_TYPE_Higher education',
       'REG_CITY_NOT_WORK_CITY', 'NAME_HOUSING_TYPE_With parents',
       'FLAG_EMP_PHONE', 'DAYS_ID_PUBLISH', 'EXT_SOURCE_2',
       'REGION_RATING_CLIENT_W_CITY', 'CODE_GENDER_M', 'ORGANIZATION_TYPE_XNA',
       'FLAG_DOCUMENT_3', 'LIVE_CITY_NOT_WORK_CITY', 'DAYS_EMPLOYED',
       'DAYS_BIRTH', 'CODE_GENDER_F', 'REG_CITY_NOT_LIVE_CITY',
       'AMT_TOTAL_RECEIVABLE', 'REGION_POPULATION_RELATIVE',
       'DAYS_LAST_PHONE_CHANGE', 'NAME_INCOME_TYPE_Pensioner',
       'AMT_GOODS_PRICE', 'AMT_BALANCE', 'DEF_60_CNT_SOCIAL_CIRCLE',
       'DAYS_REGISTRATION', 'SK_ID_CURR', 'AMT_CREDIT'],
      dtype='object')

In [109]:
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

In [110]:
df_train.to_csv("./outputs/preprocessed_data/train_data.csv")
df_test.to_csv("./outputs/preprocessed_data/test_data.csv")